# 000_Petitions_Data_Collection

In [2]:
# libraries must be installed before importing
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import requests
import time
import os

In [4]:
##method for creating a soup object 
def create_soup(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text,"html.parser")
    return soup

In [81]:
##method for getting intial petition info
def get_info(url):
    soup= create_soup(url)
    info = ""
    for paragraph1 in soup.find("p").findNext("p").findNext("p").findNext("p").findNext("p").get_text():
        info = info+paragraph1
    return info

In [82]:
##method for getting more petition info 
def get_more_info(url):
    soup= create_soup(url)
    more_info = ""
    
    for paragraph2 in soup.find("p").findNext("p").findNext("p").findNext("p").findNext("p").findNext("p").get_text():
        more_info= more_info+paragraph2
    #some petitions have no more info and contain this str. Return none 
    ##may have to include more validations here
    if "This petition is closed" in more_info:
        return None
    else:
        return more_info

In [7]:
###find the link for the transcript 
#return None if not
def find_tscrpt(url):
    soup = create_soup(url)
    
    for link in soup.findAll('a', href=True, text='Read the transcript'):
        return link['href']
        
      

In [8]:
##given the url for a transcript use this function to return text relating to them
def read_tscrpt(url):
    soup= create_soup(url)
    text = soup.find(class_='content-container')
    if text != None:
        data = list()
        for txt in text.find_all('p'):
            data.append(txt.get_text())
        data = " ".join(data)
        return data
    else:
        return None
   

In [15]:

def appendToDf(df):
    #create lists to hold variables 
    lst_info= []
    lst_more_info = []
    lst_tscrpt = []
    df['information']=''
    df['more_info']=''
    df['transcript']=''
    size = df.URL.size
    for i in range(size):
        url = df.URL[i]
        turl= find_tscrpt(url)
        if turl != None:
            x= get_info(url)
            y= get_more_info(url)
            z = read_tscrpt(turl)
        else:
            x= get_info(url)
            y= get_more_info(url)
            z= None
        lst_info.append(x)
        lst_more_info.append(y)
        lst_tscrpt.append(z)
        ###append lists to columns in data frame
        df.transcript[i]= lst_tscrpt[i]
        df.information[i]  = lst_info[i]
        df.more_info[i] = lst_more_info[i]
        if i%100== 0:
            print(i)
    return df

## 001 - Load in CSVs from downloaded from https://petition.parliament.uk

In [18]:
### load in petitons from 2010-15 and 2015-17 and 2017+ csvs to dfs 
all_petitions_current = pd.read_csv("closed-petitions.csv")
all_petitions_15_17= pd.read_csv("published-petitions-2015–2017.csv")
all_petitions_10_15 = pd.read_csv("published-petitions-2010–2015.csv")

## 002 - Append 3 additional columns for all closed petitions submitted during current government

In [64]:
# view length of data 
len(all_petitions_current)

4013

In [66]:
## append information and transcript to each row 
all_petitions_current = appendToDf(all_petitions_current)

/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [67]:
# inspect length of dataframe after append
len(all_petitions_current)

4013

In [68]:
# inspect dataframe looking at head after append
all_petitions_current.head()

,Petition,URL,State,Signatures Count,information,more_info,transcript
0,Make British Sign Language part of the Nationa...,https://petition.parliament.uk/petitions/200000,closed,35203,"Around 50,000 people in the UK use British Sig...",None,\n \r\n4.30 pm\n I draw Members’ attention to ...
1,Lobster Pots and Small Craft Safety – time to ...,https://petition.parliament.uk/petitions/200001,closed,10770,"We, the undersigned, urge the Minister of Stat...",This petition is sponsored by the Cruising Ass...,None
2,All high-rise tower blocks over 30 metres must...,https://petition.parliament.uk/petitions/200003,closed,271,Sprinklers save the lives of tower block resid...,"In 2013, the Coroner of the Shirley Towers fir...",None
3,Hold a referendum on the final Brexit deal,https://petition.parliament.uk/petitions/200004,closed,145119,"We, the undersigned, call upon HM Government t...",The referendum would allow for three options:,"\n \r\n4.30 pm\n I beg to move, That this Hou..."
4,Give all British citizens living abroad the ri...,https://petition.parliament.uk/petitions/200005,closed,10314,About 1 million British citizens living abroad...,We believe that all British citizens deserve g...,None


## 003- Append 3 additional columns for all closed petitions submitted between 2015-17

In [ ]:
# inspect length of dataframe
len(all_petitions_15_17)

In [87]:
# inspect dataframe looking at head 
all_petitions_15_17.head()

,Petition,URL,State,Signatures Count,information,more_info,transcript
0,Introduce a rate increase cap on pay TV pricin...,https://petition.parliament.uk/archived/petiti...,closed,22,It now costs £65 a month for somebody to subsc...,"For comparison, it’s possible to get a season ...",None
1,Impose a heavy extra tax on foreign buyers of ...,https://petition.parliament.uk/archived/petiti...,closed,383,British citizens are being priced out of almos...,Imposing a tax may sound like a harsh and gran...,None
2,Hold a referendum on electoral reform with the...,https://petition.parliament.uk/archived/petiti...,closed,4767,A majority of the country believes that 37% of...,In 1992 New Zealand held a referendum in which...,None
3,Make the 'Steam' refund policy the law for all...,https://petition.parliament.uk/archived/petiti...,closed,94,Current regulations treat games as movies when...,Steam Refund Policy\nhttp://store.steampowered...,None
4,Ban unpaid internships,https://petition.parliament.uk/archived/petiti...,closed,438,There is a lack of standardisation and oversig...,Some employers and middlemen can legally charg...,None


In [88]:
## append information and transcript to each row 
all_petitions_15_17 = appendToDf(all_petitions_15_17)

/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900


In [89]:
# inspect length of dataframe after append
len(all_petitions_15_17)

10950

In [90]:
# inspect dataframe looking at head after append
all_petitions_15_17.head()

,Petition,URL,State,Signatures Count,information,more_info,transcript
0,Introduce a rate increase cap on pay TV pricin...,https://petition.parliament.uk/archived/petiti...,closed,22,It now costs £65 a month for somebody to subsc...,"For comparison, it’s possible to get a season ...",None
1,Impose a heavy extra tax on foreign buyers of ...,https://petition.parliament.uk/archived/petiti...,closed,383,British citizens are being priced out of almos...,Imposing a tax may sound like a harsh and gran...,None
2,Hold a referendum on electoral reform with the...,https://petition.parliament.uk/archived/petiti...,closed,4767,A majority of the country believes that 37% of...,In 1992 New Zealand held a referendum in which...,None
3,Make the 'Steam' refund policy the law for all...,https://petition.parliament.uk/archived/petiti...,closed,94,Current regulations treat games as movies when...,Steam Refund Policy\nhttp://store.steampowered...,None
4,Ban unpaid internships,https://petition.parliament.uk/archived/petiti...,closed,438,There is a lack of standardisation and oversig...,Some employers and middlemen can legally charg...,None


## 004 - Append 3 additional columns for all closed petitions submitted between 2010-2015

In [91]:
# inspect size of dataframe
len(all_petitions_10_15)

32873

In [92]:
# inspect dataframe looking at head 
all_petitions_10_15.head()

,Petition,URL,State,Signatures Count
0,resign,https://petition.parliament.uk/archived/petiti...,closed,1849
1,Repeal the Digital Economy Act,https://petition.parliament.uk/archived/petiti...,closed,1331
2,Lets leave the European Union,https://petition.parliament.uk/archived/petiti...,closed,220
3,Airbrushed Images in the Media,https://petition.parliament.uk/archived/petiti...,closed,147
4,Reduce spending and taxes.,https://petition.parliament.uk/archived/petiti...,closed,15


In [93]:
## append information and transcript to each row 
all_petitions_10_15= appendToDf(all_petitions_10_15)

/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/donalmallon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [94]:
# inspect size of dataframe after append
len(all_petitions_10_15)

32873

In [96]:
# inspect dataframe size after append
all_petitions_10_15.head()

,Petition,URL,State,Signatures Count,information,more_info,transcript
0,resign,https://petition.parliament.uk/archived/petiti...,closed,1849,"We, the people of the United Kingdom of Great ...",None,None
1,Repeal the Digital Economy Act,https://petition.parliament.uk/archived/petiti...,closed,1331,We the public are not satisfied with the proce...,"Repeal the Digital Economy Act, and ensure any...",None
2,Lets leave the European Union,https://petition.parliament.uk/archived/petiti...,closed,220,I would like the Government to leave the Europ...,None,None
3,Airbrushed Images in the Media,https://petition.parliament.uk/archived/petiti...,closed,147,This petition calls on the Government to intro...,This petition believes this must be done to sc...,None
4,Reduce spending and taxes.,https://petition.parliament.uk/archived/petiti...,closed,15,The house of commons should instruct the Treas...,"Areas that should be considered include, but a...",None


## 005- Combine three petitions info dataframes

In [116]:
##combine three different dataframes
frames =[all_petitions_10_15,all_petitions_15_17, all_petitions_current] 
petitions_info = pd.concat(frames, ignore_index=True )
petitions_info.head()

In [5]:
# consider only petitions that have recieved over 20 signatures 
mask = petitions_info['Signatures Count'] >20
petitions_info = petitions_info.loc[mask].reset_index(drop = True)
petitions_info.shape

(20961, 7)

In [6]:
petitions_info.to_pickle('petitions_info.pickle')

### 006 - Create data frame of petitions info which doesn't take up as much memory for gui application. This data frame will only include the petition tile , url and signature count.

In [7]:
petitions_info = pd.read_pickle("petitions_info.pickle")

In [17]:
petitions_info.head(100)

,Petition,URL,State,Signatures Count,information,more_info,transcript
0,resign,https://petition.parliament.uk/archived/petiti...,closed,1849,"We, the people of the United Kingdom of Great ...",None,None
1,Repeal the Digital Economy Act,https://petition.parliament.uk/archived/petiti...,closed,1331,We the public are not satisfied with the proce...,"Repeal the Digital Economy Act, and ensure any...",None
2,Lets leave the European Union,https://petition.parliament.uk/archived/petiti...,closed,220,I would like the Government to leave the Europ...,None,None
3,Airbrushed Images in the Media,https://petition.parliament.uk/archived/petiti...,closed,147,This petition calls on the Government to intro...,This petition believes this must be done to sc...,None
4,That funding be restored to HEIs and tuition f...,https://petition.parliament.uk/archived/petiti...,closed,307,"Reforms to Higher Education are damaging, regr...","Investment in education bolsters the economy, ...",None
5,End mass immigration,https://petition.parliament.uk/archived/petiti...,closed,11156,We the undersigned call on the government to r...,None,None
6,A life sentence served,https://petition.parliament.uk/archived/petiti...,closed,58,we the undersigned call on the government to e...,None,None
7,Votes At 16 - Lower The Voting Age In All UK E...,https://petition.parliament.uk/archived/petiti...,closed,2203,We want our political system to recognise the ...,A 16 year old can:\n◦give full consent to medi...,None
8,Legalise cannabis,https://petition.parliament.uk/archived/petiti...,closed,28507,A desire for the House of Commons to give a co...,A need for Parliament to discuss the consequen...,None
9,Smoking Ban Review,https://petition.parliament.uk/archived/petiti...,closed,207,The smoking ban has lead to economic misery fo...,At the moment there are enough pubs for people...,None


In [9]:
short_petitions_info= petitions_info[['Petition', 'URL', 'Signatures Count']].copy()

In [10]:
short_petitions_info.to_pickle('short_petitions_info.pickle')

In [18]:
short_petitions_info.shape

(20961, 3)